Key Features:-

Comprehensive Applicant Details: Includes demographics, financial data, and

verification indicators.

Derived Financial Metrics: Such as Debt-to-Income Ratio to measure repayment capability.

Target Label: Loan_Approved column (1 = Approved, 0 = Denied) — useful for machine learning models.

Balanced Attributes: Mix of numeric, categorical, and binary variables suitable for both statistical and ML analysis.

In [ ]:
# ---- Import libraries ----
import sqlite3
import pandas as pd
import random

In [ ]:
conn = sqlite3.connect("loan_eligibility.db")  # creates the file if not present

In [ ]:
# ---- Create table ----
conn.execute("""
CREATE TABLE IF NOT EXISTS Customer_Data (
    Customer_ID TEXT PRIMARY KEY,
    Name TEXT,
    Age INTEGER,
    Gender TEXT,
    Marital_Status TEXT,
    Employment_Type TEXT,
    Monthly_Income DECIMAL(10,2),
    Loan_Amount_Requested DECIMAL(10,2),
    Credit_Score INTEGER,
    Existing_EMI DECIMAL(10,2),
    Loan_Tenure_Years INTEGER,
    Dependents INTEGER,
    Bank_Statement_Verified TEXT,
    Voice_Verified TEXT,
    Document_Verified TEXT,
    Debt_to_Income_Ratio DECIMAL(4,2),
    Total_Liabilities DECIMAL(10,2),
    Past_Defaults INTEGER,
    Region TEXT,
    Loan_Purpose TEXT,
    Eligibility_Score DECIMAL(3,2),
    Loan_Approved TINYINT(1)
);
""")

In [ ]:
# ---- Generate random data ----
names = ["Amit", "Priya", "Nishtha", "Rahul", "Sneha", "Rohan", "Isha", "Kiran", "Tanya", "Arjun"]
genders = ["M", "F"]
marital_statuses = ["Single", "Married"]
employment_types = ["Job", "Business", "Self-employed"]
regions = ["Delhi", "Mumbai", "Bangalore", "Chennai", "Kolkata", "Hyderabad"]
loan_purposes = ["Home", "Education", "Car", "Personal", "Business"]

data = []
for i in range(1, 101):  # 100 rows
    name = random.choice(names) + " " + random.choice(["Sharma", "Singh", "Patel", "Verma", "Kumar"])
    gender = random.choice(genders)
    age = random.randint(21, 60)
    marital = random.choice(marital_statuses)
    emp = random.choice(employment_types)
    income = random.randint(30000, 150000)
    loan_amt = random.randint(100000, 1000000)
    credit = random.randint(300, 900)
    emi = random.randint(2000, 30000)
    tenure = random.randint(1, 10)
    dep = random.randint(0, 4)
    bank_ver = random.choice(["Yes", "No"])
    voice_ver = random.choice(["Yes", "No"])
    doc_ver = random.choice(["Yes", "No"])
    debt_ratio = round(emi / income, 2)
    liabilities = random.randint(50000, 500000)
    defaults = random.randint(0, 3)
    region = random.choice(regions)
    purpose = random.choice(loan_purposes)
    eligibility = round(random.uniform(0.3, 0.95), 2)
    approved = 1 if eligibility > 0.65 and credit > 600 else 0
    customer_id = f"C{i:03d}"

    data.append((customer_id, name, age, gender, marital, emp, income, loan_amt, credit, emi, tenure, dep,
                 bank_ver, voice_ver, doc_ver, debt_ratio, liabilities, defaults, region, purpose, eligibility, approved))


In [ ]:
# ---- Insert data ----
conn.executemany("INSERT OR IGNORE INTO Customer_Data VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", data)
conn.commit()


In [ ]:
# ---- Check a few rows ----
df = pd.read_sql_query("SELECT * FROM Customer_Data LIMIT 10", conn)
display(df)

,Customer_ID,Name,Age,Gender,Marital_Status,Employment_Type,Monthly_Income,Loan_Amount_Requested,Credit_Score,Existing_EMI,...,Bank_Statement_Verified,Voice_Verified,Document_Verified,Debt_to_Income_Ratio,Total_Liabilities,Past_Defaults,Region,Loan_Purpose,Eligibility_Score,Loan_Approved
0,C001,Rohan Singh,21,M,Married,Business,48122,701743,896,26653,...,Yes,No,No,0.55,247952,2,Hyderabad,Home,0.33,0
1,C002,Tanya Kumar,36,F,Married,Job,90494,864822,440,3588,...,Yes,No,Yes,0.04,393160,0,Delhi,Personal,0.65,0
2,C003,Sneha Singh,46,F,Single,Business,127446,804413,608,5095,...,Yes,No,Yes,0.04,287727,2,Chennai,Education,0.47,0
3,C004,Tanya Singh,57,M,Single,Business,124572,591019,563,27927,...,No,Yes,Yes,0.22,149597,0,Bangalore,Education,0.39,0
4,C005,Amit Patel,53,F,Married,Business,59344,992266,437,28827,...,No,No,No,0.49,329539,2,Mumbai,Home,0.95,0
5,C006,Rohan Sharma,45,M,Married,Job,105990,572876,711,10083,...,No,Yes,No,0.10,361817,1,Kolkata,Car,0.37,0
6,C007,Amit Verma,32,M,Married,Job,119014,988551,627,3248,...,No,No,No,0.03,60658,2,Hyderabad,Car,0.90,1
7,C008,Tanya Kumar,41,F,Single,Self-employed,58275,848642,806,16581,...,No,No,Yes,0.28,200392,2,Bangalore,Business,0.47,0
8,C009,Amit Verma,27,F,Married,Self-employed,99720,156064,341,5288,...,No,Yes,Yes,0.05,106863,3,Hyderabad,Education,0.38,0
9,C010,Amit Sharma,48,F,Single,Job,63172,569699,790,13984,...,Yes,Yes,Yes,0.22,123637,0,Mumbai,Education,0.94,1


In [ ]:
# ---- Export to CSV ----
df_full = pd.read_sql_query("SELECT * FROM Customer_Data", conn)
df_full.to_csv("Customer_Data.csv", index=False)
print("✅ 100 rows inserted successfully and exported to Customer_Data.csv!")

✅ 100 rows inserted successfully and exported to Customer_Data.csv!


In [ ]:

conn.close()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!cp Customer_Data.csv /content/drive/MyDrive/


Mounted at /content/drive


Possible Insights

Identify top factors influencing loan approval (e.g., credit score, income, or employment type).

Analyze the relationship between debt-to-income ratio and loan decisions.

Visualize approval rates by region or loan purpose.

Build and evaluate classification models (e.g., Logistic Regression, Random Forest).

Use for data cleaning, feature engineering, and SQL queries practice.

The dataset contains 100 synthetic customer records designed for loan eligibility prediction and SQL practice. Data covers demographics, financial history, and verification details suitable for exploratory analysis and machine learning tasks.